# Superlative Cities

#### Outline

1. Find cities with most and least severe car accidents
2. Feature Analysis for each city
    - Identify most important and most common weather features
    - Identify most important and most common infrastructure features
    (using random forests?)
3. Plots for each city
    - Show where accidents occur and how severe they are on top of a city map kind of
    - Histograms of accident severity for most common condition per weather feature
    - Histogram of accident severity per infrastructure type side by side per city

In [1]:
import pandas as pd
import numpy as np

data = pd.read_csv('US_Accidents_March23.csv')

Data cleaning

In [4]:
drop_columns = ['ID', 'Source', 'Start_Time', 'End_Time', 
                 'End_Lat', 'End_Lat', 'End_Lng', 'Distance(mi)', 'Description',
                 'Zipcode', 'Country', 'Timezone', 'Airport_Code', 'Weather_Timestamp', 
                 'Amenity', 'Civil_Twilight', 'Nautical_Twilight', 'Astronomical_Twilight']

df_dropped = data.drop(drop_columns, axis=1)

In [ ]:
cities = df_dropped.groupby('City')
city_stats = cities['Severity'].agg(['mean', 'count']).sort_values('mean')

,mean,count
City,,
St Leonard,1.0,1
Sullivan City,1.0,1
Center Conway,1.0,1
Caneyville,1.0,1
Gracewood,1.0,1
...,...,...
Christopher,4.0,2
Slade,4.0,1
Lake Andes,4.0,1


In [ ]:
city_counts = df_dropped['City'].value_counts()
city_counts.describe()
# decide which cities to drop before determinig superlatives,
# what should the cutoff be?

count     13678.000000
mean        565.005191
std        4147.357347
min           1.000000
25%           6.000000
50%          35.000000
75%         201.000000
max      186917.000000
Name: count, dtype: float64

In [ ]:
# get two cities and drop/fill columns with NaNs